In [168]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
import re

# Manthan's Data

In [170]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}
base_url = ['https://www.espncricinfo.com/series/ipl-2016-968923/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2017-1078425/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2018-1131611/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2019-1165643/match-schedule-fixtures-and-results',
            'https://www.espncricinfo.com/series/ipl-2020-21-1210595/match-schedule-fixtures-and-results']
soups = []
for s in base_url:
    response = requests.get(s,headers=headers)
    soups.append(BeautifulSoup(response.text,'html.parser'))
    print(response.status_code)

200
200
200
200
200


In [171]:
allseasonmatches = []

for i,soup in enumerate(soups) :
    seasonmatch = []
    anchors = soup.find_all('a',class_='ds-inline-flex ds-items-start ds-leading-none')
    for anchor in anchors :
        link = anchor.get('href')
        link = urljoin(base_url[i],link)
        if 'live-cricket-score' in link :
            seasonmatch.append(link)
    allseasonmatches.append(seasonmatch)
allseasonmatches

[['https://www.espncricinfo.com/series/ipl-2016-968923/mumbai-indians-vs-rising-pune-supergiants-1st-match-980901/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/kolkata-knight-riders-vs-delhi-daredevils-2nd-match-980903/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/kings-xi-punjab-vs-gujarat-lions-3rd-match-980905/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/royal-challengers-bangalore-vs-sunrisers-hyderabad-4th-match-980907/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/kolkata-knight-riders-vs-mumbai-indians-5th-match-980909/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/gujarat-lions-vs-rising-pune-supergiants-6th-match-980911/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/delhi-daredevils-vs-kings-xi-punjab-7th-match-980913/live-cricket-score',
  'https://www.espncricinfo.com/series/ipl-2016-968923/sunrisers

Columns to fill
1. Match type
2. POTM
3. Venue
4. Team1
5. Team2
6. Toss Winner
7. Toss Decision
8. Winner
9. Win margin
10. Result type(number of wickets left or run difference)
11. Target runs

## Match Details

In [174]:
'''
# asmd - all season match data
# smd - season match data
asmd = []
for i,seasonmatch in enumerate(allseasonmatches):
    smd = []
    for j,matchlink in enumerate(seasonmatch):
        
        match_data = {
                'Match Type': '',
                'POTM': '',
                'Venue': '',
                'Team1': '',
                'Team2': '',
                'Toss_Winner': '',
                'Toss_Decision': '',
                'Winner': '',
                'Win_Margin': '',
                'Result_Type': '',
                'Target_Runs': ''
        }

        match = requests.get(matchlink,headers=headers)
        #print(match.status_code)
        soup_match = BeautifulSoup(match.text,'html.parser')

        driver = webdriver.Chrome()
        driver.get(matchlink)

        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        elements = soup.find_all('div', class_='ds-text-tight-m ds-font-regular ds-text-typo-mid3')[0].get_text(strip=True)
        potmscrape = soup.find_all('span', class_='ds-text-tight-m ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer')[0].get_text(strip=True)
        teamscrape = soup.find_all('span',class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
        tossscrape = soup.find_all('td',class_='ds-text-typo')[0].find('span',class_='ds-text-tight-s ds-font-regular').get_text(strip=True)

        driver.quit()

        #  Match type, POTM and Venue
        
        venue = elements.split(',')[1].strip()
        match_data['Venue'] = venue

        matching_word = next((word for word in ['Final','Qualifier','Eliminator'] if word in elements), None)
        
        if matching_word:
            match_data['Match Type'] = matching_word
        else:
            match_data['Match Type'] = 'League'
        match_data['POTM'] = potmscrape
        
        # Teams
        
        match_data['Team1'] = teamscrape[0].get_text(strip=True)
        match_data['Team2'] = teamscrape[1].get_text(strip=True)
        
        # Toss Winner and Decision
        
        match_data['Toss_Winner'] = tossscrape.split(',')[0]
        if 'bat' in tossscrape :    
            match_data['Toss_Decision'] = 'bat'
        else :
            match_data['Toss_Decision'] = 'field'
        
        ## Win Details
        
        winscrape = soup.find('p',class_='ds-text-tight-s ds-font-medium ds-truncate ds-text-typo').find('span').get_text(strip=True)
        match_data['Winner'] = winscrape.split()[0]
        if 'wicket' in winscrape : 
            match_data['Result_Type'] = 'wicket'
        else :
            match_data['Result_Type'] = 'runs'
        
        match_data['Win_Margin'] = re.findall(r'\d+', winscrape)[0]
        
        ## Target Runs
        
        trunscrape = soup.find_all('div',class_='ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap')[0].find('strong').get_text(strip=True)
        match_data['Target_Runs'] = trunscrape.split('/')[0]
        smd.append(match_data)
    asmd.append(smd)
'''

IndexError: list index out of range

In [178]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")  # run in headless mode

driver = webdriver.Chrome(options=chrome_options)

In [179]:
asmd = []
for i, seasonmatch in enumerate(allseasonmatches):
    smd = []
    for j, matchlink in enumerate(seasonmatch):
        match_data = {
            'Match Type': '',
            'POTM': '',
            'Venue': '',
            'Team1': '',
            'Team2': '',
            'Toss_Winner': '',
            'Toss_Decision': '',
            'Winner': '',
            'Win_Margin': '',
            'Result_Type': '',
            'Target_Runs': ''
        }

        # Optional: You can still use requests to check the status if needed
        match_response = requests.get(matchlink, headers=headers)
        if match_response.status_code != 200:
            print(f"Failed to retrieve {matchlink}")
            continue

        # Use Selenium to load the JavaScript-rendered content
        driver.get(matchlink)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # Extract elements as before:
        try:
            elements = soup.find_all('div', class_='ds-text-tight-m ds-font-regular ds-text-typo-mid3')[0].get_text(strip=True)
        except IndexError:
            elements = ''
        try:
            potmscrape = soup.find_all('span', class_='ds-text-tight-m ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer')[0].get_text(strip=True)
        except IndexError:
            potmscrape = ''
        try:
            teamscrape = soup.find_all('span', class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
        except IndexError:
            teamscrape = []
        try:
            tossscrape = soup.find_all('td', class_='ds-text-typo')[0].find('span', class_='ds-text-tight-s ds-font-regular').get_text(strip=True)
        except IndexError:
            tossscrape = ''
        # Process match details:
        if elements:
            try:
                venue = elements.split(',')[1].strip()
            except IndexError:
                venue = ''
            match_data['Venue'] = venue
            matching_word = next((word for word in ['Final', 'Qualifier', 'Eliminator'] if word in elements), None)
            match_data['Match Type'] = matching_word if matching_word else 'League'
        match_data['POTM'] = potmscrape
        
        if teamscrape and len(teamscrape) >= 2:
            match_data['Team1'] = teamscrape[0].get_text(strip=True)
            match_data['Team2'] = teamscrape[1].get_text(strip=True)
        else:
            match_data['Team1'] = match_data['Team2'] = ''
        
        if tossscrape:
            match_data['Toss_Winner'] = tossscrape.split(',')[0]
            match_data['Toss_Decision'] = 'bat' if 'bat' in tossscrape.lower() else 'field'
        
        try:
            winscrape = soup.find('p', class_='ds-text-tight-s ds-font-medium ds-truncate ds-text-typo').find('span').get_text(strip=True)
            match_data['Winner'] = winscrape.split()[0]
            match_data['Result_Type'] = 'wicket' if 'wicket' in winscrape.lower() else 'runs'
            match_data['Win_Margin'] = re.findall(r'\d+', winscrape)[0]
        except (IndexError, AttributeError):
            match_data['Winner'] = match_data['Result_Type'] = match_data['Win_Margin'] = ''
        
        try:
            trunscrape = soup.find_all('div', class_='ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap')[0].find('strong').get_text(strip=True)
            match_data['Target_Runs'] = trunscrape.split('/')[0]
        except (IndexError, AttributeError):
            match_data['Target_Runs'] = ''
        
        smd.append(match_data)
    asmd.append(smd)

driver.quit()

In [191]:
frames = []
for smd in asmd :
    df = pd.DataFrame(smd)
    frames.append(df)
result = pd.concat(frames)
result

,Match Type,POTM,Venue,Team1,Team2,Toss_Winner,Toss_Decision,Winner,Win_Margin,Result_Type,Target_Runs
0,,,,,,,,,,,
1,,,,,,,,,,,
2,,,,,,,,,,,
3,,,,,,,,,,,
4,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
55,,,,,,,,,,,
56,,,,,,,,,,,
57,,,,,,,,,,,
58,,,,,,,,,,,
